## Helper functions

### Visualization helpers

In [ ]:
import numpy as np
from IPython import display
import matplotlib.pyplot as plt
import time
import imageio
%matplotlib inline


def init_fig(env):
    plt.figure()
    plt.title(env.spec.id)
    plt.axis('off')
    return plt.imshow(env.render(mode='rgb_array'))

def update_fig(img, env, step, reward, done, stl_rob, stl_violated):
    title = plt.title(
        f"{env.spec.id}\n" +
        f"Step: {step} | Reward: {reward:.3f} | Done: {done}\n" +
        f"STL: {stl_rob[0]:.3f} | Violated: {stl_violated}"
    )
    if done or stl_violated:
        plt.setp(title, color='r')
    else:
        plt.setp(title, color='k')
    img.set_data(env.render(mode='rgb_array'))
    fig = plt.gcf()
    display.display(fig)
    display.clear_output(wait=True)
    
    return np.asarray(fig.canvas.buffer_rgba())

def visual_episode(env, next_action, compute_stl_rob, is_stl_violated,
                   max_episode_steps=100, reset_to=None, visualize_in_notebook=True,
                   sleep=0.01, save_gif=None, model_reset=None):
    if reset_to is not None:
        obs = env.reset_to(reset_to)
    else:
        obs = env.reset()
    
    if model_reset is not None:
        model_reset()
    
    if save_gif is not None:
        gif = []
    else:
        gif = None

    # initialize figure in notebook
    if visualize_in_notebook:
        img = init_fig(env)

    space = env.observation_space
    total_reward = 0.0
    obs_records = [obs]
    
    for step in range(1, max_episode_steps+1):
        action = next_action(obs)
        obs, reward, done, info = env.step(action)
        total_reward += reward
        obs_records.append(np.clip(obs, space.low, space.high))

        if visualize_in_notebook:
            rob = compute_stl_rob(np.asarray(obs_records))
            fig_data = update_fig(img, env, step, total_reward, done, rob, is_stl_violated(rob))
            if save_gif is not None:
                gif.append(fig_data)
        else:
            env.render()

        if sleep > 0.0:
            time.sleep(sleep)
    
    if save_gif is not None:
        imageio.mimsave(save_gif, [data for data in gif], fps=10)

In [ ]:
def visual_deviated_env(x, reset_to, save_gif=None):
    env = build_env(x)
    env.seed(seed)
    visual_episode(env, next_action, compute_stl_rob, is_stl_violated, max_episode_steps=max_episode_steps,
                   reset_to=reset_to, save_gif=save_gif, model_reset=model_reset)
    env.close()

### Functions for scaling the variable

In [ ]:
def scale(x, bounds):
    """Scale the input numbers in [0, 1] to the range of each variable"""
    return bounds[:, 0] + x * (bounds[:, 1] - bounds[:, 0])

def normalize(x_scaled, bounds):
    return (x_scaled - bounds[:, 0]) / (bounds[:, 1] - bounds[:, 0])

### STL robustness computation helpers

In [ ]:
import signal_tl as stl

def compute_stl_rob(records):
    if records.ndim == 2:
        records = np.array([records])
    
    time_index = np.arange(records.shape[1])
    robs = []
    for i in range(len(records)):
        trace = build_stl_trace(records[i], time_index)
        rob = stl.compute_robustness(phi, trace)
        robs.append(rob.at(0))
    
    return np.array(robs)

### STL falsification helpers

#### Falsification by random sampling

In [ ]:
def random_sample(env):
    space = env.observation_space
    episode_obs_records = []

    for episode in range(max_episodes):
        obs = env.reset()
        
        if model_reset is not None:
            model_reset()
        
        obs_records = [obs]
        
        for step in range(max_episode_steps):
            action = next_action(obs)
            obs, reward, done, info = env.step(action)
            obs_records.append(np.clip(obs, space.low, space.high))

        episode_obs_records.append(obs_records)

    return np.array(episode_obs_records)

def falsification_by_random_sample(x):
    """
    x is the deviation parameter in its original domain
    """
    env = build_env(x)
    env.seed(seed)
    records = random_sample(env)
    env.close()
    return compute_stl_rob(records).min()

def scaled_falsification_by_random_sample(x):
    """
    Wrapped falsification function where input x is in [0, 1] and should be scaled to the original domain.
    """
    return falsification_by_random_sample(scale(x, dev_bounds))

#### Falsification by CMA

In [ ]:
import cma

def episode_by_init(env, x):
    space = env.observation_space
    obs = env.reset_to(x)
    obs_records = [obs]
    
    if model_reset is not None:
        model_reset()
    
    for step in range(max_episode_steps):
        action = next_action(obs)
        obs, reward, done, info = env.step(action)
        obs_records.append(np.clip(obs, space.low, space.high))
    
    return np.array(obs_records)

def falsification_by_CMA(x):
    env = build_env(x)
    env.seed(seed)
    
    def objective(y):
        record = episode_by_init(env, y)
        return compute_stl_rob(record)[0]
    
    y0 = np.zeros(len(falsify_range)) # TODO: https://lig-kobe-grenoble.imag.fr/wp-content/uploads/2018/03/05.-Session-4-presentation-1-Thao-Dang.pdf
    sigma0 = 0.2
    min_x, es = cma.fmin2(
        lambda y: objective(scale(y, falsify_range)),
        y0,
        sigma0,
        {'bounds': [0.0, 1.0], 'maxfeval': max_episodes, 'ftarget': stl_falsify_target,
         'verbose': -9, 'seed': seed}
    )
    
    env.close()
    return es.result.fbest, scale(min_x, falsify_range)

def scaled_falsification_by_CMA(x):
    fbest, _ = falsification_by_CMA(scale(x, dev_bounds))
    return fbest

#### Compare two sampling methods

In [ ]:
def compare_falsification(trials):
    for x in np.random.rand(trials, len(dev_bounds)):
        est1 = scaled_falsification_by_random_sample(x)
        est2 = scaled_falsification_by_CMA(x)
        if est1 < est2:
            print(est1, est2, "Random Sample")
        else:
            print(est1, est2, "CMA")

### Miscs

In [ ]:
def evaluate_dev(x, gif):
    robustness, x_init = falsification_by_CMA(x)
    visual_deviated_env(x, x_init, gif)
    print("Deviation:", x)
    print("Deviation distance:", deviation_dist(x))
    print("Initial parameter:", x_init)
    print("Min robustness value:", robustness)

### Robustness computation helpers

In [ ]:
import os

if not os.path.exists('gifs'):
    os.mkdir('gifs')

In [ ]:
# l-2 norm distance, the input x is in the original domain
def deviation_dist(x):
    """The variables of the objective function are normalized to [0, 1)"""
    return np.sqrt(np.sum((x - x_original) ** 2))

# wrapped objective function where input x is in [0, 1] and should be scaled to the original domain
def scaled_deviation_dist(x):
    return deviation_dist(scale(x, dev_bounds))

In [ ]:
def dev_violates_stl():
    x0 = normalize(x_original, dev_bounds)
    sigma0 = 0.4

    _, es = cma.fmin2(
        scaled_falsification_by_CMA,
        x0,
        sigma0,
        {'bounds': [0.0, 1.0], 'ftarget': stl_falsify_target, 'tolstagnation': 0,
         'tolx': 1e-5, 'timeout': timeout * 60},
    )
    print(es.result)
    return scale(es.result.xbest, dev_bounds)

In [ ]:
def min_dev_violates_stl():
    min_dev = np.inf
    min_x = None
    for i in range(num_tries):
        print(f'\n================ Trial {i} ==============>')

        x0 = normalize(x_original, dev_bounds)
        sigma0 = 0.2

        cfun = cma.ConstrainedFitnessAL(
            scaled_deviation_dist,
            lambda x: [scaled_falsification_by_CMA(x)],
            find_feasible_first=True
        )
        _, es = cma.fmin2(
            cfun,
            x0,
            sigma0,
            {'bounds': [0.0, 1.0], 'tolstagnation': 0, 'tolx': 1e-5,
             'timeout': timeout / num_tries * 60},
            callback=cfun.update
        )

        print(es.result)
        if cfun.best_feas.info is None:
            cfun.find_feasible(es)
        print(cfun.best_feas.info)

        x = cfun.best_feas.info['x']
        dev = deviation_dist(scale(x, dev_bounds))
        if dev < min_dev:
            min_dev = dev
            min_x = x
    
    return scale(min_x, dev_bounds)

In [ ]:
def max_safe_dev():
    max_dev = -np.inf
    max_x = None
    for i in range(num_tries):
        print(f'\n================ Trial {i} ==============>')

        x0 = normalize(x_original, dev_bounds)
        sigma0 = 0.2

        cfun = cma.ConstrainedFitnessAL(
            lambda x: -scaled_deviation_dist(x), # maximize distance
            lambda x: [-scaled_falsification_by_CMA(x)], # robustness >= 0
        #     find_feasible_first=True # assume the initial is already feasible
        )
        _, es = cma.fmin2(
            cfun,
            x0,
            sigma0,
            {'bounds': [0.0, 1.0], 'tolstagnation': 0, 'tolx': 1e-5, 'timeout': timeout / num_tries * 60},
            callback=cfun.update
        )

        print(es.result)
        if cfun.best_feas.info is None:
            cfun.find_feasible(es)
        print(cfun.best_feas.info)

        x = cfun.best_feas.info['x']
        dev = deviation_dist(scale(x, dev_bounds))
        if dev > max_dev:
            max_dev = dev
            max_x = x
    
    return scale(max_x, dev_bounds)

## Config your environment and agent

### TODO: Define the environment

In [ ]:
# define the range of the deviation parameters
# dev_bounds = 

# define the range of the variables to falsify
# falsify_range = 

# define the deviation parameter value for the original environment (no deviation)
# x_original = 

In [ ]:
def build_env(x):
    """
    Create the environment given the deviation parameter in its original (non-normalized) domain.
    """
    pass

### TODO: Define Agent

In [ ]:
# define the action generation function of the given agent
# next_action = 

# some agents (like PID) needs to be reset for each run
# model_reset = None

### TODO: Define STL property

In [ ]:
def build_stl_prop():
    pass

def build_stl_trace(records, time_index):
    pass

def is_stl_violated(rob):
    pass

phi = build_stl_prop()

### TODO: Config global options and functions

In [ ]:
# seed = 

# define the max epsiode steps when evaluating a given environment
# max_episode_steps = 200

# define the max episodes to run when falsifying a given environment (under some deviation)
# max_episodes = 100

# define the target STL robustness value when falsification, i.e., CMA would stop search
# when an instance of such value is found.
# stl_falsify_target = -np.inf

# timeout in minutes
# timeout = 30

# Could change this value for different trials within the total timeout
# num_tries = 3

## Robustness analysis starts here!

### The agent should be safe under the original environment by falsification

In [ ]:
evaluate_dev(x_original, "gifs/default_env.gif")

### CMA for minimizing the STL robustness value given the bounded deviation

In [ ]:
x = dev_violates_stl()

In [ ]:
evaluate_dev(x, "gifs/counterexample.gif")

### CMA-ES for minimizing the deviation s.t. the STL is violated.

In [ ]:
x = min_dev_violates_stl()

In [ ]:
evaluate_dev(x, "gifs/min_dev_counterexample.gif")

### [Optional] CMA for finding the maximum deviation s.t. the agent is safe

In [ ]:
x = max_safe_dev()

In [ ]:
evaluate_dev(x, "gifs/max_safe_deviation.gif")